# PSKY Analysis - Conditional Value At Risk
**Expected loss give that the loss is equal or greater than the value at risk** <br>
**Conditional Value at Risk** - expected tail-loss/shortfall in the worst 1% of all cases<br>
**Value at Risk** - is the minimum loss in the worst 1% of all cases<br>

* Takes into consideration very extreme negative outcomes in the laft-tail


* 252 Trading Days in a Year
* 63 Business Days in a Quarter
* Simulating pirces over a multi-period is called a stochastic process
* Parametric reutns - assumes normality of returns (i.e. normaly distributed)
    * Assumes Daily returns are independent from each other
* Bootstrapping Method - does not assume normality of returns and works with the actual distribution of the histoircal returns

#### Problem Definition: 
Simulate the conditional expected loss over a period of one quarter that will occur with 1% probability. Use and comapre Parametric simulation & Bootstrap simulation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
plt.style.use("seaborn-v0_8")

In [2]:
class FinancialInstrument():
    ''' Class for analyzing Financial Instruments like stocks.

    Attributes
    ==========
    ticker: str
        ticker symbol with which to work with
    start: str
        start date for data retrieval
    end: str
        end date for data retrieval

    Methods
    =======
    get_data:
        retrieves daily price data (from yahoo finance) and prepares the data
    log_returns:
        calculates log returns
    plot_prices:
        creates a price chart
    plot_returns:
        plots log returns either as time series ("ts") or histogram ("hist")
    set_ticker:
        sets a new ticker
    mean_return:
        calculates mean return
    std_returns:
        calculates the standard deviation of returns (risk)
    annualized_perf:
        calculates annulized return and risk
    '''
    
    def __init__(self, ticker, start, end):
        self.ticker = ticker
        self.start = start
        self.end = end
        self.get_data()
        self.log_returns()
    
    def __repr__(self): 
        return "FinancialInstrument(ticker = {}, start = {}, end = {})".format(self.ticker, 
                                                                               self.start, self.end)
    def get_data(self):
        ''' retrieves (from yahoo finance) and prepares the data
        '''
        raw = yf.download(self.ticker, self.start, self.end, multi_level_index = False).Close.to_frame() # new from yfinance 0.2.48
        raw.rename(columns = {"Close":"price"}, inplace = True)
        self.data = raw
        
    def log_returns(self):
        '''calculates log returns
        '''
        self.data["log_returns"] = np.log(self.data.price/self.data.price.shift(1))
        
    def plot_prices(self):
        ''' creates a price chart
        '''
        self.data.price.plot(figsize = (12, 8))
        plt.title("Price Chart: {}".format(self.ticker), fontsize = 15)
    
    def plot_returns(self, kind = "ts"):
        ''' plots log returns either as time series ("ts") or histogram ("hist")
        '''
        if kind == "ts":
            self.data.log_returns.plot(figsize = (12, 8))
            plt.title("Returns: {}".format(self.ticker), fontsize = 15)
        elif kind == "hist":
            self.data.log_returns.hist(figsize = (12, 8), bins = int(np.sqrt(len(self.data))))
            plt.title("Frequency of Returns: {}".format(self.ticker), fontsize = 15)
    
    def set_ticker(self, ticker = None):
        '''sets a new ticker
        '''
        if ticker is not None:
            self.ticker = ticker
            self.get_data()
            self.log_returns()
            
    def mean_return(self, freq = None):
        '''calculates mean return
        '''
        if freq is None:
            return self.data.log_returns.mean()
        else:
            resampled_price = self.data.price.resample(freq).last()
            resampled_returns = np.log(resampled_price / resampled_price.shift(1))
            return resampled_returns.mean()
    
    def std_returns(self, freq = None):
        '''calculates the standard deviation of returns (risk)
        '''
        if freq is None:
            return self.data.log_returns.std()
        else:
            resampled_price = self.data.price.resample(freq).last()
            resampled_returns = np.log(resampled_price / resampled_price.shift(1))
            return resampled_returns.std()
        
    def annualized_perf(self):
        '''calculates annulized return and risk
        '''
        mean_return = round(self.data.log_returns.mean() * 252, 3)
        risk = round(self.data.log_returns.std() * np.sqrt(252), 3)
        print("Return: {} | Risk: {}".format(mean_return, risk))

In [3]:
stock = FinancialInstrument(ticker = "PSKY", start = "2000-01-01", 
                            end =  "2025-12-31" ) # instantiation

[*********************100%***********************]  1 of 1 completed


In [4]:
# exporting data to csv
#stock.data.to_csv("PSKY_data.csv")

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
np.set_printoptions(precision=4, suppress=True)

In [6]:
returns = stock.data.log_returns.dropna().values

In [7]:
returns

array([-0.0358, -0.0082, -0.0063, ..., -0.0059,  0.0154, -0.011 ],
      shape=(5046,))

In [8]:
#returns.size

## VaR for $1M USD Portfolio of PSKY - Parametric Method

In [11]:
def path_simul_param(daily_returns, days, IO, sims = 10000, seed = 123):
    
    mean = np.mean(daily_returns)
    std = np.std(daily_returns)
    days = int(days)

    np.random.seed(seed)  # for reproducibility
    ret = np.random.normal(loc=mean, scale=std, size=(days * sims)).reshape(sims, days)

    paths = (ret + 1).cumprod(axis=1) * IO
    paths = np.hstack((np.ones(sims).reshape(sims, 1)*IO, paths))

    return paths

In [9]:
sims = 1000000
IO = 1000000

In [12]:
paths = path_simul_param(returns, days=63, IO=IO, sims=sims)

In [13]:
paths.shape

(1000000, 64)

In [14]:
# final portfolio values
final_p = paths[:, -1]

In [15]:
final_p

array([1142104.8555,  895408.6048, 1000178.2745, ...,  866229.6124,
        965278.7837, 1098609.6156], shape=(1000000,))

In [16]:
prob = 0.01

In [17]:
np.percentile(final_p, prob*100)

np.float64(566046.7524998498)

In [18]:
np.percentile(final_p, prob*100) - IO

np.float64(-433953.24750015023)

In [19]:
# values lower than the 1st percentile
tail_p = final_p[final_p < np.percentile(final_p, prob*100)]

### Conditional VaR Parametric

In [ ]:
# Conditional VaR Parametric
tail_p.mean() - IO

np.float64(-475407.5682494936)

## VaR for $1M USD Portfolio of PSKY - Bootstrap Method

In [21]:
def path_simul_bootsstr(daily_returns, days, IO, sims = 10000, seed = 123):

    days  = int(days)

    np.random.seed(seed)
    ret = np.random.choice(daily_returns, size=(days * sims), replace=True).reshape(sims, days)

    paths = (ret + 1).cumprod(axis = 1) * IO
    paths = np.hstack((np.ones(sims).reshape(sims, 1) * IO, paths))

    return paths

In [22]:
paths = path_simul_bootsstr(returns, days = 63, IO = IO, sims = sims)

In [23]:
final_b = paths[:,-1]

In [24]:
final_b

array([1046461.7471, 1032758.6814,  978947.8453, ..., 1357569.9547,
        761510.4535, 1460189.361 ], shape=(1000000,))

In [25]:
np.percentile(final_b, prob*100)

np.float64(536288.6145153864)

In [26]:
np.percentile(final_b, prob*100) - IO

np.float64(-463711.3854846136)

In [27]:
tail_b = final_b[final_b < np.percentile(final_b, prob*100)]
tail_b

array([402812.5831, 471826.151 , 503664.1917, ..., 484409.4956,
       526161.1462, 524716.4825], shape=(10000,))

### Conditional VaR Bootstrap

In [29]:
# Conditional VaR Bootstrap
tail_b.mean() - IO

np.float64(-517165.7292981868)